<a href="https://colab.research.google.com/github/Adnan-Aziz/PythonPractice/blob/master/GA_8Queens_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

In [2]:
class DNA:

    def __init__(self, dimension):
        self.dimension = dimension
        self.gene = np.random.randint(0, dimension, size=(dimension,))
        self.fitness = 0
        self.chessboard = np.zeros(shape=(self.dimension, self.dimension), dtype=int)

    def calc_fitness(self):
        total_temp_sum = 0
        total_penalty = 0

        for i in range(self.dimension):
            self.chessboard[self.gene[i], i] = 1

        for i in range(self.dimension):
            temp_sum, penalty = self.diag_ne(self.gene[i], i)
            total_temp_sum += temp_sum
            total_penalty += penalty
            if total_temp_sum == 7:
                break

            temp_sum, penalty = self.diag_nw(self.gene[i], i)
            total_temp_sum += temp_sum
            total_penalty += penalty
            if total_temp_sum == 7:
                break

            temp_sum, penalty = self.diag_sw(self.gene[i], i)
            total_temp_sum += temp_sum
            total_penalty += penalty
            if total_temp_sum == 7:
                break

            temp_sum, penalty = self.diag_se(self.gene[i], i)
            total_temp_sum += temp_sum
            total_penalty += penalty
            if total_temp_sum == 7:
                break

        for i in range(self.dimension):
            temp_sum = sum(self.chessboard[i])
            if temp_sum == 0:
                total_penalty += 4
            elif temp_sum > 1:
                total_temp_sum *= temp_sum

        self.fitness = total_temp_sum + total_penalty ** 2

    def diag_nw(self, i, j):
        temp_sum = 0
        penalty = 0
        prev_val = -1
        while i >= 0 and j >= 0:
            val = self.chessboard[i, j]
            temp_sum += val
            if prev_val == 1 and val == 1:
                penalty += 2
            prev_val = val
            i -= 1
            j -= 1
        temp_sum -= 1
        return temp_sum, penalty

    def diag_ne(self, i, j):
        temp_sum = 0
        penalty = 0
        prev_val = -1
        while i >= 0 and j < self.dimension:
            val = self.chessboard[i, j]
            temp_sum += val
            if prev_val == 1 and val == 1:
                penalty += 2
            prev_val = val
            i -= 1
            j += 1
        temp_sum -= 1
        return temp_sum, penalty

    def diag_sw(self, i, j):
        temp_sum = 0
        penalty = 0
        prev_val = -1
        while i < self.dimension and j >= 0:
            val = self.chessboard[i, j]
            temp_sum += val
            if prev_val == 1 and val == 1:
                penalty += 2
            prev_val = val
            i += 1
            j -= 1
        temp_sum -= 1
        return temp_sum, penalty

    def diag_se(self, i, j):
        temp_sum = 0
        penalty = 0
        prev_val = -1
        while i < self.dimension and j < self.dimension:
            val = self.chessboard[i, j]
            temp_sum += val
            if prev_val == 1 and val == 1:
                penalty += 2
            prev_val = val
            i += 1
            j += 1
        temp_sum -= 1
        return temp_sum, penalty

    def crossover(self, other):
        child = DNA(self.dimension)
        # midpoint = random.randint(0, self.dimension)
        midpoint = (int)(self.dimension/2)
        child.gene[:midpoint] = self.gene[:midpoint]
        child.gene[midpoint:] = other.gene[midpoint:]
        return child

    def mutate(self, mutation_rate):
        for i in range(self.dimension):
            if random.random() <= mutation_rate:
                index_i = random.randint(0, self.dimension-1)
                index_j = random.randint(0, self.dimension - 1)
                # print(index)
                self.gene[index_i], self.gene[index_j] = self.gene[index_j],self.gene[index_i]
                self.gene[i] = random.randint(0, self.dimension-1)
                # self.gene[i] = i
        self.chessboard = np.zeros(shape=(self.dimension, self.dimension), dtype=int)

    def get_gene(self):
        return self.gene

    def get_chessboard(self):
        for i in range(self.dimension):
            self.chessboard[self.gene[i], i] = 1
        return self.chessboard

    def get_fitness(self):
        return self.fitness

    def __str__(self):
        return self.gene

In [3]:
from typing import List, Any, Union
import random



class Population:
    # population: Union[List[DNA], Any]

    def __init__(self, dimension, size, mutation_rate):
        self.population = [None] * size
        self.mutation_rate = mutation_rate
        self.generation = 0
        self.finished = False
        self.target_score = 0
        self.size = size
        self.best_score = 10000
        self.best_dna=[None]
        self.avg_fitness=0
        for i in range(self.size):
            self.population[i] = DNA(dimension)
        self.calc_fitness()

    def calc_fitness(self):
        total_fitness = 0
        for i in range(self.size):
            self.population[i].calc_fitness()
            total_fitness += self.population[i].get_fitness()
        self.avg_fitness = total_fitness/self.size

    def generate(self):
        new_population = [None] * self.size

        for i in range(self.size):
            parent_a = self.accept_reject()
            parent_b = self.accept_reject()
            child = parent_a.crossover(parent_b)
            child.mutate(self.mutation_rate)
            new_population[i] = child
        self.population = new_population
        self.generation += 1

    def accept_reject(self):
        max_fitness = 0
        min_fitness = 0
        for i in range(self.size):
            if self.population[i].get_fitness() > max_fitness:
                max_fitness = self.population[i].get_fitness()
            if self.population[i].get_fitness() < min_fitness:
                min_fitness = self.population[i].get_fitness()

        for i in range(self.size):
            index = random.randint(0, self.size-1)
            rand_fitness = random.randint(min_fitness, max_fitness)
            if self.population[index].get_fitness() < rand_fitness:
                return self.population[index]
        return self.population[index]

    def evaluate(self):
        for i in range(self.size):
            # print(self.population[i].get_fitness())
            if self.best_score > self.population[i].get_fitness():
                self.best_score = self.population[i].get_fitness()
                self.best_dna = self.population[i]
        # print(f'Inside evaluate best score {self.best_score} , target score {self.target_score}')
        if self.best_score == self.target_score:
            self.finished = True

    def get_best(self):
        return self.best_score

    def get_average_fitness(self):
        return self.avg_fitness

    def get_current_generation(self):
        return self.generation

    def is_finished(self):
        return self.finished

    def display_info(self):
        print(f'Generation: {self.generation} Avg Score : {self.avg_fitness}  '
              f'Elite : {self.best_dna.get_gene()} Best Score : {self.best_score} ')
        # print(f'Average fitness score : {self.avg_fitness}')
        # print(f'Best Element : {self.best_dna.get_gene()}')
        # print(f'Best Score : {self.best_score}')
        # print(self.population)
        # print('ChessBoard')
        # print(self.best_dna.get_chessboard())


In [4]:
population = Population(8, 50, 0.02)
population.calc_fitness()
thres_hold = 1000
i=0
while not population.is_finished():
    i+=1 
    population.calc_fitness()
    population.evaluate()
    population.generate()
    population.display_info()
    if population.get_current_generation() > thres_hold:
        population = Population(8, 50, 0.02)
    if i > 10000:
        break

print("Closing")
population.display_info()
print(population.best_dna.get_chessboard())

Generation: 1 Avg Score : 359.02  Elite : [4 0 3 0 4 1 5 2] Best Score : 72 
Generation: 2 Avg Score : 277.66  Elite : [5 1 3 0 6 2 7 1] Best Score : 28 
Generation: 3 Avg Score : 253.96  Elite : [5 1 3 0 6 2 7 1] Best Score : 28 
Generation: 4 Avg Score : 232.78  Elite : [3 6 0 4 7 5 1 2] Best Score : 26 
Generation: 5 Avg Score : 227.96  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 6 Avg Score : 236.34  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 7 Avg Score : 238.46  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 8 Avg Score : 173.7  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 9 Avg Score : 226.48  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 10 Avg Score : 202.62  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 11 Avg Score : 178.4  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 12 Avg Score : 240.6  Elite : [3 6 0 4 4 1 5 2] Best Score : 20 
Generation: 13 Avg Score : 226.14  Elite : [3 6 0 4 4 1 5 2] Best Score : 20